In [73]:
%reload_ext autoreload
%autoreload 2

In [74]:
import os
from pathlib import Path
import polars as pl
# import pandas as pd
import requests
from tqdm import tqdm
from notebook_utils import insert_parent_in_path
insert_parent_in_path()

from src.etl.pipeline import fetch_raw_data
from src.etl.transform import  transform_raw_data

# NYC Schema

In [56]:
from src.etl.models import NYCPickupHourlySchema

In [62]:
NYCPickupHourlySchema._cast_datetime_columns(pl.DataFrame)

(polars.dataframe.frame.DataFrame, ['pickup_datetime_hour'])

In [55]:
initial_data = {
    "key": ["A_2023_01", "B_2023_01", "C_2023_01"],
    "pickup_datetime_hour": ["2023-01-01 10:00:00", "2023-01-01 10:00:00", "2023-01-01 10:00:00"],
    "num_pickups": [10, 20, 30],
    "pickup_location_id": [1, 2, 3]
}

# Create new data with some overlapping keys but different values
new_data = {
    "key": ["B_2023_01", "C_2023_01", "D_2023_01"],
    "pickup_datetime_hour": ["2023-01-01 10:00:00", "2023-01-01 10:00:00", "2023-01-01 10:00:00"],
    "num_pickups": [25, 35, 40],  # Changed values for B and C
    "pickup_location_id": [2, 3, 4]
}




In [67]:
initial_data = {
    "key": ["A_2023_01", "B_2023_01", "C_2023_01"],
    "pickup_datetime_hour": ["2023-01-01 10:00:00", "2023-01-01 10:00:00", "2023-01-01 10:00:00"],
    "num_pickups": [10, 20, 30],
    "pickup_location_id": [1, 2, 3]
}


(
    pl.DataFrame(initial_data, schema=NYCPickupHourlySchema.get_type_mapping())
)

AttributeError: 'str' object has no attribute 'tzinfo'

# ETL single file

In [75]:
df = fetch_raw_data(year=2024, month=10)
df.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-10-01 00:30:44,2024-10-01 00:48:26,1,3.0,1,"""N""",162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0
1,2024-10-01 00:12:20,2024-10-01 00:25:25,1,2.2,1,"""N""",48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0
1,2024-10-01 00:04:46,2024-10-01 00:13:52,1,2.7,1,"""N""",142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0
1,2024-10-01 00:12:10,2024-10-01 00:23:01,1,3.1,1,"""N""",233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0
1,2024-10-01 00:30:22,2024-10-01 00:30:39,1,0.0,1,"""N""",262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0


In [77]:
clean_data = transform_raw_data(df, 2024, 10)
clean_data.head()

2025-02-09 18:15:16,106 - src.etl.transform - INFO - Validate data for year: 2024 and month: 10
2025-02-09 18:15:16,107 - src.etl.transform - INFO - Total records: 3833771
2025-02-09 18:15:16,107 - src.etl.transform - INFO - Records deleted: 40
2025-02-09 18:15:16,109 - src.etl.transform - INFO - Percentage: 100.00%
2025-02-09 18:15:16,183 - src.etl.transform - INFO - Aggregating data to hourly frequency


key,pickup_datetime_hour,num_pickups,pickup_location_id
str,datetime[μs],i32,i32
"""2024-10-01 00:00:00.000000-1""",2024-10-01 00:00:00,0,1
"""2024-10-01 00:00:00.000000-2""",2024-10-01 00:00:00,0,2
"""2024-10-01 00:00:00.000000-3""",2024-10-01 00:00:00,0,3
"""2024-10-01 00:00:00.000000-4""",2024-10-01 00:00:00,2,4
"""2024-10-01 00:00:00.000000-5""",2024-10-01 00:00:00,0,5


# Saving to Local Repo

In [69]:
from src.repository.local_repo import LocalRepository
repo = LocalRepository()
repo.create_tables()

2025-02-09 18:13:15,362 - src.repository.local_repo - INFO - Created nyc_trips.main.pickup_hourly table


In [78]:
repo.upsert_pickup_data(clean_data)

2025-02-09 18:15:37,120 - src.repository.local_repo - INFO - No existing data found


In [83]:
from src.etl.pipeline import file_etl
from src.repository.duck_repo import DuckDBRepository



duck_repo = DuckDBRepository(db_mode='LOCAL')

duck_repo.create_tables()


file_etl(duck_repo, 2024, 10)

2025-02-09 18:29:53,152 - src.repository.duck_repo - INFO - Running DB in mode LOCAL
2025-02-09 18:29:53,167 - src.repository.duck_repo - INFO - Created nyc_trips.main.pickup_hourly table
2025-02-09 18:29:56,530 - src.etl.transform - INFO - Validate data for year: 2024 and month: 10
2025-02-09 18:29:56,531 - src.etl.transform - INFO - Total records: 3833771
2025-02-09 18:29:56,532 - src.etl.transform - INFO - Records deleted: 40
2025-02-09 18:29:56,533 - src.etl.transform - INFO - Percentage: 100.00%
2025-02-09 18:29:56,585 - src.etl.transform - INFO - Aggregating data to hourly frequency
2025-02-09 18:29:58,434 - src.repository.duck_repo - INFO - Upserted into dwh.main.pickup_hourly


# DuckDB Repository

In [25]:
from datetime import datetime, date 



from_date = date(2023,3,5)
to_date = date(2024,7,29)

def truncate_start_of_month(x:date):
    return date(x.year, x.month, 1)

def generate_list_of_months(from_date:date, to_date:date):
    from_date = truncate_start_of_month(from_date)
    to_date = truncate_start_of_month(to_date)

    current_date = from_date
    dates = []
    while current_date <= to_date:
        dates.append(current_date)
        if current_date.month == 12:
            current_date = date(current_date.year+1, 1,1)
        else:
            current_date = date(current_date.year, current_date.month+1, 1)
    return dates
    
    


[datetime.date(2023, 3, 1),
 datetime.date(2023, 4, 1),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 6, 1),
 datetime.date(2023, 7, 1),
 datetime.date(2023, 8, 1),
 datetime.date(2023, 9, 1),
 datetime.date(2023, 10, 1),
 datetime.date(2023, 11, 1),
 datetime.date(2023, 12, 1),
 datetime.date(2024, 1, 1),
 datetime.date(2024, 2, 1),
 datetime.date(2024, 3, 1),
 datetime.date(2024, 4, 1),
 datetime.date(2024, 5, 1),
 datetime.date(2024, 6, 1),
 datetime.date(2024, 7, 1)]

# Download files

In [9]:

# def download_file_from_source_into_raw_folder(year:int, month:int) -> Path:
#     file = RAW_DATA_DIR / Path(FILE_PATTERN.format(year=year, month=month))
#     url = BASE_URL.format(year=year, month=month)
#     response = requests.get(url)
#     response.raise_for_status()
#     if response.status_code == 200:
#         logger.info(f"Downloading file from {url} to {file}")
#         with open(file, "wb") as f:
#             f.write(response.content)
#     return file




# Validate a file
- Timestamp are within the file month. e.g, a file named `2020-07-01.csv` should have all timestamps within July 2020.

In [10]:
# df = pl.read_parquet(path)
# df.head()

In [11]:
path = download_file_from_source_into_raw_folder(2020, 1)

df = (
    validate_file(path, 2020,1)
    .pipe(aggregate_pickup_into_timeseries_data, 2020,1)
    .pipe(generate_surrogate_key)
)



df.head()

2024-12-02 07:51:02,258 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:03,779 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:03,782 - src.data - INFO - Total records: 6405008
2024-12-02 07:51:03,784 - src.data - INFO - Records deleted: 212
2024-12-02 07:51:03,785 - src.data - INFO - Percentage: 100.00%


key,pickup_datetime_hour,pickup_location_id,num_pickups
str,datetime[ns],i16,i16
"""2020-01-01 00:00:00.000000000-…",2020-01-01 00:00:00,1,0
"""2020-01-01 00:00:00.000000000-…",2020-01-01 00:00:00,2,0
"""2020-01-01 00:00:00.000000000-…",2020-01-01 00:00:00,3,1
"""2020-01-01 00:00:00.000000000-…",2020-01-01 00:00:00,4,57
"""2020-01-01 00:00:00.000000000-…",2020-01-01 00:00:00,5,0


In [12]:
file_etl(2020, 1)

2024-12-02 07:51:11,765 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:12,694 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:12,695 - src.data - INFO - Total records: 6405008
2024-12-02 07:51:12,697 - src.data - INFO - Records deleted: 212
2024-12-02 07:51:12,698 - src.data - INFO - Percentage: 100.00%
2024-12-02 07:51:13,806 - src.data - INFO - Deleted file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:15,205 - src.dwh - INFO - Upserted C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\processed\yellow_tripdata_2020-01.parquet into d

# DuckDB

In [13]:
PARENT_DIR

WindowsPath('C:/Users/selaf/OneDrive/Documentos/dev/real-ml-course/taxi_demand_predictor')

In [14]:
# import duckdb


# DATABASE_URL = DATA_DIR / "dwh.duckdb"
# db = duckdb.connect(database=str(DATABASE_URL))


# def create_pickup_table(db: duckdb.DuckDBPyConnection):
#     db.execute("DROP TABLE IF EXISTS dwh.main.pickup_hourly;")
#     db.execute(
#         """
#         CREATE TABLE dwh.main.pickup_hourly (
#             key STRING PRIMARY KEY
#             , pickup_datetime_hour TIMESTAMP
#             , pickup_location_id SMALLINT
#             , num_pickup SMALLINT
#         );
#         """
#     )
    
    

    # return statement



In [15]:
file_etl(2020, 2)

2024-12-02 07:51:18,823 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-02.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-02.parquet
2024-12-02 07:51:20,458 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-02.parquet
2024-12-02 07:51:20,459 - src.data - INFO - Total records: 6299367
2024-12-02 07:51:20,460 - src.data - INFO - Records deleted: 303
2024-12-02 07:51:20,461 - src.data - INFO - Percentage: 100.00%
2024-12-02 07:51:20,664 - src.data - INFO - Deleted file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-02.parquet
2024-12-02 07:51:21,596 - src.dwh - INFO - Upserted C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\processed\yellow_tripdata_2020-02.parquet into d

In [16]:
batch_etl(year=2020)

2024-12-02 07:51:21,695 - src.data - INFO - Downloading data for year 2020


  0%|          | 0/12 [00:00<?, ?it/s]

2024-12-02 07:51:24,256 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:25,149 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:25,150 - src.data - INFO - Total records: 6405008
2024-12-02 07:51:25,150 - src.data - INFO - Records deleted: 212
2024-12-02 07:51:25,151 - src.data - INFO - Percentage: 100.00%
2024-12-02 07:51:25,515 - src.data - INFO - Deleted file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-01.parquet
2024-12-02 07:51:26,320 - src.dwh - INFO - Upserted C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\processed\yellow_tripdata_2020-01.parquet into d

  8%|▊         | 1/12 [00:04<00:50,  4.61s/it]

2024-12-02 07:51:28,678 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-02.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-02.parquet
2024-12-02 07:51:29,560 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-02.parquet
2024-12-02 07:51:29,561 - src.data - INFO - Total records: 6299367
2024-12-02 07:51:29,562 - src.data - INFO - Records deleted: 303
2024-12-02 07:51:29,562 - src.data - INFO - Percentage: 100.00%
2024-12-02 07:51:29,780 - src.data - INFO - Deleted file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-02.parquet
2024-12-02 07:51:30,503 - src.dwh - INFO - Upserted C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\processed\yellow_tripdata_2020-02.parquet into d

...

2024-12-02 07:51:52,922 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-11.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-11.parquet
2024-12-02 07:51:53,124 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-11.parquet
2024-12-02 07:51:53,125 - src.data - INFO - Total records: 1509000
2024-12-02 07:51:53,125 - src.data - INFO - Records deleted: 112
2024-12-02 07:51:53,126 - src.data - INFO - Percentage: 99.99%
2024-12-02 07:51:53,169 - src.data - INFO - Deleted file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-11.parquet
2024-12-02 07:51:54,053 - src.dwh - INFO - Upserted C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\processed\yellow_tripdata_2020-11.parquet into dw

 92%|█████████▏| 11/12 [00:32<00:02,  2.77s/it]

2024-12-02 07:51:55,777 - src.data - INFO - Downloading file from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-12.parquet to C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-12.parquet
2024-12-02 07:51:55,968 - src.data - INFO - Validation for file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-12.parquet
2024-12-02 07:51:55,969 - src.data - INFO - Total records: 1461898
2024-12-02 07:51:55,970 - src.data - INFO - Records deleted: 35
2024-12-02 07:51:55,971 - src.data - INFO - Percentage: 100.00%
2024-12-02 07:51:56,005 - src.data - INFO - Deleted file: C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\raw\yellow_tripdata_2020-12.parquet
2024-12-02 07:51:56,856 - src.dwh - INFO - Upserted C:\Users\selaf\OneDrive\Documentos\dev\real-ml-course\taxi_demand_predictor\data\processed\yellow_tripdata_2020-12.parquet into dw

100%|██████████| 12/12 [00:35<00:00,  2.93s/it]

2024-12-02 07:51:56,860 - src.data - INFO - Data for year 2020 has been downloaded and validated
